# Contexte

Ce rapport étant le premier adressé à Thalès depuis le début de ma thèse qui a démarré le 1er Novembre 2023, une remise en contexte semble être la bienvenue.

Mon sujet de thèse consiste à trouver une technique d'ajustement en temps réel des paramètres d'un composant optique actif, appelé "Kernel Nuller", permettant la détection d'exoplanètes a haut contraste par annulation de la lumière de son étoile.

L'architecture sur laquelle j’effectue ces recherches prend 4 télescopes en comme source d'information. Une première série de MMI (Multi Mode Interferometer) vient combiner ces 4 sources de sorte à former une sortie contenant des interférences constructives des 4 entrées, puis trois sorties constituées de couple d'entrées en opposition de phase, aussi appelé "sorties nulles". Un second étage de MMI, appelé "Split & Mix", vient ensuite s'ajouter pour déphaser et recombiner les trois sorties nulles en trois paires de "sorties sombres", constitués des quatre signaux d'entré séparés d'une phase $\frac{\pi}{4}$.

Entre chacune des rangées de MMI se trouvent des retardateurs (ou "phase shifters") sur lesquels il est possible d'agir électriquement afin de jouer sur la phase de la lumière traversant la gaine optique. Ces retardateurs nous permettent de compenser les diverses perturbations subies par le système. On peut séparer ces retardateurs en deux groupes :
- $P_1 \rightarrow P_4$ sont des retardateurs dynamiques, ils devront être ajustés en temps réel pour compenser des perturbation atmosphérique ou défauts dans la correction du front d'onde.
- $P_5 \rightarrow P_{14}$ sont des retardateurs statiques qui permettent de compenser des défauts intrinsèque au composant. Ces paramètres peuvent donc être ajustés en amont sur un banc optique.

![image](https://github.com/Leirof/Tunable-Kernel-Nulling/assets/12165342/b709c245-6c4e-4ae8-8d3a-e72b55b0db29)

# Objectif actuel : Caractérisation & point de fonctionnement idéal

En premier objectif, on aimerait comprendre comment chaque paramètre influe sur les différentes sorties et être capable de déterminer la valeur à appliquer aux retardateurs passifs afin d'emmener le composant au plus proche de son point de fonctionnement idéal.

Pour cela, j'ai modélisé l'action du composant a travers 3 représentations différentes : 
- Symbolique : Chaque MMI peut être représenté par une matrice 2x2. J'implémente alors l'expression de ces composants dans Sympy, un outil de calcul symbolique, et je multiplie les différentes matrices en respectant l'ordre de propagation de sorte à avoir une matrice final correspondant au Kernel-Nuller dans son ensemble. Cette représentation est pratique car elle donne directement les fonctions de transfert `f(x1, ..., x4) -> (bright, dark1, ..., dark6)` mais contient une grande quantité de termes et est donc pas très pratique à manipuler.
- Numérique : De la même façon, j'exprime la matrice de chaque MMI et je propage un vecteur contenant les amplitudes complexes appliquées en entrées. La fonction de transfert globale n'est donc pas explicitée mais cette représentation est bien plus facile a manipuler pour voir l'impact de chaque élément.
- Analytique : Afin de répondre à certaines questions sur le modèle numérique et/ou symbolique, j'ai simplifié l'expression des différents composants, sans passer par une représentation matricielle et sans prendre en compte les multiples sources de perturbation. Cette représentation à l'avantage d'être très simple à établir "à la main" et à implémenter dans un petit programme python. Elle est cependant très incomplète et ne sera donc pas conservée. Il ne s'agit que d'une représentation simplifiée pour aider à mettre au point et/ou corriger les deux premières représentations.

Pour le moment, je m’attèle a faire concorder ces différentes représentations. Celle symbolique par exemple donne des paires de sortie sombre (identifié par les couleurs sur l'image ci-dessous) identiques pratiquement en tout point, avec un facteur de phase globale comme seule différence (termes soulignés). Ce n'est évidemment pas censé être le cas, 2 des 4 composantes est censé conserver la même phase dans une paire de sortie sombre.

![image](https://github.com/Leirof/Tunable-Kernel-Nulling/assets/12165342/ff675dd9-2550-40ae-ae68-3489e2dda1c8)

Le modèle numérique quant à lui semble donner des résultats cohérents, mais pour s'en assurer, il faudrait que je parvienne à obtenir les mêmes résultats en implémentant directement la fonction de transfert du composant.

Sur le plot ci-dessous, on peut voir l'intensité observée sur chacune des sorties en fonction du décalage induit par $P_1$ et $P_2$, en mettant en entrée 4 flux parfaitement en phase et d'intensité 1 (unité arbitraire).

![image](https://github.com/Leirof/Tunable-Kernel-Nulling/assets/12165342/9f7a5459-8c2e-4ab7-bcdc-6258e7790370)

Pour m'assurer de la validité de ces résultats, j'ai donc implémenté numériquement la fonction de transfert obtenue via la représentation analytique. Après quelques complications (erreurs d'implémentations), cette dernière m'a donné des résultats très similaire :

![image](https://github.com/Leirof/Tunable-Kernel-Nulling/assets/12165342/38d36281-5dea-42ab-bff9-9f9a0e34104a)

Malheureusement, si les cartes semblent similaires, les amplitudes ne le sont pas et semblent incohérente sur ces derniers plots (généré par la fonction de transfert directement).

# Perspective

Il est nécessaire de parvenir a réconcilier ces différentes représentation pour s'assurer qu'elles soient correct. De là, on pourra vraiment étudier l'impact de chacun des paramètres puis vérifier en laboratoire si le modèle donne des prédictions satisfaisantes. De là on pourra essayer de caractériser le composant en labo et déterminer les retards a appliquer sur les retardateurs statiques ($P_5 \rightarrow P_{14}$)
